In [2]:
import os
import re
import json
import tiktoken
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
DATA_DIR = "../data/raw"
cards = []

# .json파일 cards에 담기
for filename in os.listdir(DATA_DIR):
    if filename.endswith(".json"):
        file_path = os.path.join(DATA_DIR, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            card_data = json.load(f)
            cards.extend(card_data)

print(f"총 {len(cards)}개의 카드 정보를 불러왔습니다.")

총 967개의 카드 정보를 불러왔습니다.


In [ ]:
cards[0]

In [7]:
tokenizer = tiktoken.encoding_for_model("text-embedding-3-large")

In [8]:
def chunk_text(text, max_length=2000, overlap=200):
    tokens = tokenizer.encode(text)
    chunks = []

    for i in range(0, len(tokens), max_length - overlap):
        chunk_tokens = tokens[i:i + max_length]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)

        if i + max_length >= len(tokens):
            break

    return chunks

In [9]:
# 문서화 + 청킹 (덮어쓰기 방식)
texts, metadatas, ids = [], [], []

for idx, card in enumerate(cards):
    # 카드 정보 파싱
    name = card.get("name", "")
    brand = card.get("brand", "")
    url = card.get("detail_url", "")
    fee_domestic = card.get("fee_domestic", "")
    fee_global = card.get("fee_global", "")
    card_schemes = ", ".join(card.get("card_schemes", []))
    benefits = card.get("benefits", [])

    # 문서 텍스트 구성 (카드 이름 + 혜택 내용)
    benefit_texts = []
    for b in benefits:
        category = b.get("category", "")
        content = b.get("content", "")
        if category and content:
            benefit_texts.append(f"{category}: {content}")
    
    full_text = f"{name}\n" + "\n".join(benefit_texts)

    # 메타데이터 구성
    meta = {
        "name": name,
        "brand": brand,
        "fee_domestic": fee_domestic,
        "fee_global": fee_global,
        "card_schemes": card_schemes
    }

    # 청킹 → 텍스트, 메타데이터, ID 저장
    chunks = chunk_text(full_text)
    for i, chunk in enumerate(chunks):
        texts.append(chunk)
        metadatas.append(meta)
        ids.append(f"card-{idx}-chunk-{i}")

In [ ]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
chroma = Chroma(
    collection_name="card_collection",
    persist_directory="../data/processed/hcw/chroma_db",
    embedding_function=embedding_model
)

In [12]:
BATCH_SIZE = 100
for i in range(0, len(texts), BATCH_SIZE):
    chroma.add_texts(
        texts[i:i + BATCH_SIZE],
        metadatas=metadatas[i:i + BATCH_SIZE],
        ids=ids[i:i + BATCH_SIZE]
    )

In [ ]:
chroma.persist()

In [14]:
print(f"texts: {len(texts)}, metadatas: {len(metadatas)}, ids: {len(ids) if 'ids' in locals() else 'N/A'}, DB 저장 문서 수: {chroma._collection.count()}")

texts: 2415, metadatas: 2415, ids: 2415, DB 저장 문서 수: 2415


In [15]:
print(f"DB에 저장된 문서 수: {chroma._collection.count()}")

DB에 저장된 문서 수: 2415


In [16]:
query = "휴대폰 요금 자동납부 할인 카드가 궁금해"

In [17]:
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

query_vector = embedder.embed_query(query)

# Chroma 내부 collection 객체로 직접 질의
results = chroma._collection.query(
    query_embeddings=[query_vector],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [19]:
# 결과 출력
for i in range(len(results["documents"][0])):
    print(f"\n📌 결과 {i+1}")
    print(f"유사도 점수 (낮을수록 유사): {results['distances'][0][i]:.4f}")
    print(f"문서 내용:\n{results['documents'][0][i][:200]}...")
    print(f"메타데이터: {results['metadatas'][0][i]}")



📌 결과 1
유사도 점수 (낮을수록 유사): 0.7580
문서 내용:
적 기준 미충족, 자동납부 해지 등의 사유로 서비스가 적용되지 않더라도 결제일 할인 제공회차는 소진됩니다.
- 이동통신 자동납부 건은 한 달 동안 이용한 금액이 다음 달에 결제되며, 롯데카드에 접수되는 일자를 기준으로 할인 혜택이 제공됩니다.
- 남은 할인한도는 이월되지 않습니다.
- 결제 취소 건 접수 내역은 롯데카드 홈페이지(www.lottecard.c...
메타데이터: {'fee_domestic': '20,000', 'brand': '롯데카드', 'card_schemes': 'Mastercard', 'name': '모바일엔디지로카', 'fee_global': '20,000'}

📌 결과 2
유사도 점수 (낮을수록 유사): 0.8360
문서 내용:
닥터구디 T&R(티앤알) 카드
통신: 통신요금 자동이체 최대 15,000원 청구할인
- 통신요금 자동이체 가맹점: SKT, KT, LGU+, SK텔링크, LG헬로비전, U+알뜰모바일, 현대HCN, 딜라이브, 에넥스텔레콤, 플래시모바일, KT엠모바일, 프리텔레콤, 에스원 안심모바일, SK브로드밴드, 여유텔레콤, 씨엠비, KT Skylife
[자동이체 청구할인...
메타데이터: {'brand': '하나카드', 'name': '닥터구디 T&R(티앤알) 카드', 'fee_global': '25,000원', 'card_schemes': 'AMEX', 'fee_domestic': '25,000원'}

📌 결과 3
유사도 점수 (낮을수록 유사): 0.8414
문서 내용:
CLUB SK 카드
SKT: SK텔레콤 이동통신 요금 청구할인
- SK텔레콤 이동통신 요금 자동이체 설정 시, 휴대폰 기종 / 자동이체 방법에 따라 월 2천원~1만5천원까지 청구할인 (월 1회)
SK텔레콤 이동통신 요금 청구할인기준
지난달 일시불/할부 카드 이용금액40만원 이상70만원 미만70만원 이상100만원 미만100만원 이상LTE모바일5,000원10,...
메타데이터: {'name'

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
def extract_filters(user_query):
    filters = {}

    for brand in ["롯데카드", "신한카드", "하나카드", "KB국민카드", "삼성카드", "우리카드", "현대카드", "NH농협카드", "IBK기업은행", "BC카드"]:
        if brand in user_query:
            filters["brand"] = brand
            user_query = user_query.replace(brand, "")

    for scheme in ["Visa", "Mastercard", "AMEX", "JCB", "UnionPay"]:
        if scheme.lower() in user_query.lower():
            filters["card_schemes"] = scheme
            user_query = re.sub(scheme, "", user_query, flags=re.IGNORECASE)

    m = re.search(r"연회비\s*(\d{1,3}[,]?\d{3})\s*원\s*이하", user_query)
    if m:
        max_fee = m.group(1).replace(",", "")
        filters["fee_global"] = max_fee

    if len(filters) > 1:
        filter_chroma = {"$and": [{k: v} for k, v in filters.items()]}
    else:
        filter_chroma = filters

    return user_query.strip(), filter_chroma

In [22]:
query = "배달앱 혜택 있는 카드 추천해줘"
query_text, filter_meta = extract_filters(query)

In [ ]:
retriever = chroma.as_retriever(
    search_kwargs={
        "k": 5,
        **({"filter": filter_meta} if filter_meta else {})
    }
)
retrieved_docs = retriever.get_relevant_documents(query)

contexts = [doc.page_content for doc in retrieved_docs]

In [24]:
prompt = PromptTemplate.from_template("""
당신은 신용카드 혜택을 분석하여 사용자에게 가장 적합한 카드를 추천하는 전문 상담 챗봇입니다.

아래 Context에는 카드 이름, 카드사, 연회비, 주요 혜택 정보가 정리되어 있습니다.  
사용자의 질문을 바탕으로 조건에 부합하는 카드를 3개 추천해주세요.

반드시 지켜야 할 지침:

- 답변은 **반드시 context 안의 정보만** 기반으로 작성하세요.  
- 문서에 없는 정보를 추가하거나 추론하지 마세요.
- 카드별로 최소 3줄 이상의 설명을 포함해야 합니다:
  - 1줄: 카드 이름 및 브랜드
  - 2줄: 주요 혜택 요약 (카테고리 중심)
  - 3줄: 혜택 사용 조건, 예외사항 또는 한도 설명 등
- 조건에 맞는 카드가 없다면 **"조건에 맞는 카드를 찾을 수 없습니다."** 라고 답하세요.

출력 형식 예시:

1. **롯데쇼핑카드** - 롯데카드  
   - 온라인 쇼핑, 네이버페이, 쿠팡 등에서 10% 청구할인 혜택이 제공됩니다.  
   - 월 최대 1만원까지 할인되며, 전월 실적 30만원 이상 시 적용됩니다.

2. **신한배달카드** - 신한카드  
   - 배달앱(배달의 민족, 요기요, 쿠팡이츠)과 편의점(GS25, CU)에서 각각 10% 할인 혜택이 있습니다.  
   - 할인 한도는 항목별로 월 5천원이며, 전월 실적 40만원 이상이 필요합니다.

---

사용자 질문:
{question}

카드 정보(Context):
{context}
""")

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [26]:
response = qa_chain.invoke({"query": query})
print(response['query'])
print(response["result"])

배달앱 혜택 있는 카드 추천해줘
1. **요기요 삼성카드** - 삼성카드  
   - 요기요에서 10% 결제일할인 혜택을 제공합니다. 전월 이용금액에 따라 할인 한도가 달라지며, 30만원 이상 시 월 최대 6,000원, 60만원 이상 시 12,000원, 90만원 이상 시 20,000원까지 할인됩니다.  
   - 할인은 전월 이용금액 30만원 이상 시 제공되며, 발급월 + 1개월까지는 30만원 미만 실적 구간에서도 혜택이 제공됩니다. 또한, 공식 홈페이지나 앱을 통한 결제에만 적용됩니다.

2. **MY WAY 카드** - 삼성카드  
   - 대중교통에서 건당 300원 할인 혜택이 제공되며, 커피, 편의점 등에서도 유사한 혜택이 있습니다. 대중교통 이용 시, 전월 실적에 따라 최대 30건까지 할인 받을 수 있습니다.  
   - 대중교통 할인은 시내버스 및 지하철에만 적용되며, 시외버스와 고속버스는 제외됩니다. 할인은 전월 이용금액에 따라 달라지며, 커피나 편의점에서도 동일한 할인 혜택이 제공됩니다.

3. **부릉 삼성카드 BIZ** - 삼성카드  
   - 커피, 이동통신, 주유 등에서 1~3%의 빅포인트 적립 혜택이 제공됩니다. 주유소에서는 1% 적립, 커피 전문점에서는 3% 적립이 가능합니다.  
   - 적립 한도는 업종에 따라 다르며, 이동통신의 경우 자동납부건에서만 적립이 가능합니다. 적립 제외 대상에는 대중교통과 무이자할부 등이 포함되어 있습니다.


In [27]:
query_vector = embedder.embed_query(query)

results = chroma._collection.query(
    query_embeddings=[query_vector],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)

for doc, dist in zip(results["documents"][0], results["distances"][0]):
    print(f"✅ 유사도 점수: {1 - dist:.3f}")  # 코사인 거리 → 유사도 = 1 - 거리
    print(doc[:300])

✅ 유사도 점수: 0.051
요기요 삼성카드
배달앱: 서비스안내
- 요기요 10% 결제일할인
할인기준
- 전월 이용금액 30만원 이상: 월 할인한도 6,000원
- 전월 이용금액 60만원 이상: 월 할인한도 12,000원
- 전월 이용금액 90만원 이상: 월 할인한도 20,000원
이용조건
- 전월 이용금액 30만원 이상 시 제공
- 발급월+1개월까지는 전월 이용금액 30만원 미만 시에도 30만원 이상~60만원 미만 실적구간 혜택 제공(전월 이용금액 60만원 이상 시에는 해당 실적구간 혜택 제공)
할인 제외 대상
- 무이자할부, 삼성카드 할인이 적용된 일시불 
✅ 유사도 점수: 0.046
e플래티넘 롯데카드
바우처: 스페셜 기프트 - 매년 2만원 캐시백
- 발급 첫 해 : 연간 이용금액 10만원 이상 이용 시 2만원 캐시백
- 다음해부터 : 연간 이용금액 300만원 이상 및 스마일페이 가맹점 1회 이상 이용 시 2만원 캐시백
* 연회비 정상 납부 시 제공
* 신청방법: 고객센터 1588-8100 또는 홈페이지(www.lottecard.co.kr) 신청
* 이 서비스는 최초 발급월 기준으로 신청조건 충족 시 연 1회 신청 가능하며, 연회비 면제 회원 및 가족카드 회원은 제외됩니다.
* 이 서비스의 이용금액은 카드 발급
✅ 유사도 점수: 0.034
#MY WAY 카드
대중교통: 대중교통 건당 300원 할인
할인 대상 가맹점
- 버스, 지하철
지난달 실적별 월 할인 건 수
- 20만원 이상: 10건
- 60만원 이상: 20건
- 100만원 이상: 30건
제공 서비스별 건당 결제 조건
- 대중교통: 3백원 이상
※ 대중교통 할인 안내
- 대중교통 할인의 경우 월 1회 대중교통요금 청구 시 할인 건 수 적용하여 할인
- 대중교통 이용건 수 산정방식: 월 합산 대중교통요금 청구금액/ 1,200원(소수점 이하 절사)
- 대중교통 이용건 수 산정방식에 따라 산출된 이용건 수당 300원씩
✅ 유사도 점수: 0.028
카카오뱅크 삼성카드
할인: 전제가맹점
- 국내외 온오프라인 

In [ ]:
from ragas.metrics import answer_relevancy, faithfulness
from ragas.evaluation import evaluate
import pandas as pd
from datasets import Dataset

# 평가용 데이터 구성
df = pd.DataFrame([{
    "question": "연회비 1만원 이하에 스타벅스 할인되는 카드",
    "answer": response["result"],
    "contexts": contexts 
}])

# DataFrame → Dataset 변환
ragas_dataset = Dataset.from_pandas(df)

# 평가 실행
results = evaluate(
    ragas_dataset,
    metrics=[answer_relevancy, faithfulness]
)

results